In [3]:
## 此代码删去了所有的绘图，只用来计算b值。
## 计算了所有文件的b值并取平均。
## 由于线性回归的训练集选取具有随机性，所以每次的结果也不尽相同。

import os

SZtick_filenames = []

def walk(path):
    if not os.path.exists(path):
        return -1
    for root,dirs,names in os.walk(path):
        for filename in names:
            print(os.path.join(root,filename)) #路径和文件名连接构成完整路径
            SZtick_filenames.append(os.path.join(root,filename))

if __name__ == '__main__':
    walk(r'C:\Users\86157\Downloads\quantml-master (1)\quantml-master\data-15.07.08')
    # print('遍历结束！')


import h5py

b_all = 0
#dataPath = "data-15.07.08/8.9//000029.SZtick.mat"
for dataPath in SZtick_filenames:
    #dataPath = SZtick_filenames[1]
    f = h5py.File(dataPath,'r+')
    f['r1'].keys()
    import numpy as np
    import pandas as pd
    keys = ['AccTurover', 'AccVolume', 'AskAvPrice', 'AskPrice', 'AskVolume', 'BSFlag', 'BidAvPrice', 'BidPrice', 'BidVolume', 'Code', 'CurDelta', 'Date', 'Downs', 'High', 'HoldLines', 'Index', 'Interest', 'Low', 'MatchItems', 'Open', 'Position', 'PreClose', 'PrePosition', 'PreSettle', 'Price', 'Settle', 'Stocks', 'Time', 'TotalAskVolume', 'TotalBidVolume', 'TradeFlag', 'Turover', 'Ups', 'Volume', 'WindCode']
    data = []
    for key in keys:
        data.append(f['r1'][key])
    BidPrice = np.array(f['r1']['BidPrice'])
    AskPrice = np.array(f['r1']['AskPrice'])
    BidVolume = np.array(f['r1']['BidVolume'])
    AskVolume = np.array(f['r1']['AskVolume'])
    TotalBidVolume = np.array(f['r1']['TotalBidVolume'])## 添加于2.12.16.55
    TotalAskVolume = np.array(f['r1']['TotalAskVolume'])
    Price = np.array(f['r1']['Price']) 
    miu_n = BidVolume.astype(np.int) - AskVolume.astype(np.int)
    Total_miu_n = TotalBidVolume.astype(np.int) - TotalAskVolume.astype(np.int) 

    import numpy as np
    Price_change = np.zeros(Price.shape)
    Total_miu_n_change = np.zeros(Total_miu_n.shape)
    # Price_change[0,1:Y_Price-1] = Price[0,1:Y_Price-1] - Price[0,0:Y_Price-2]

    [dummy,Y_totmiun] = Total_miu_n.shape
    [dummy,Y_Price] = Price.shape

    DIST_TOTALMIU = 1
    for i in range(DIST_TOTALMIU,Y_totmiun-1): 
        Total_miu_n_change[0,i] = Total_miu_n[0,i] - Total_miu_n[0,i-DIST_TOTALMIU]
    for i in range(1,DIST_TOTALMIU-1):
        Total_miu_n_change[0,i] = Total_miu_n[0,i]

    DIST_PRICE = 5
    for i in range(DIST_PRICE,Y_Price-1): 
        Price_change[0,i] = Price[0,i] - Price[0,i-DIST_PRICE]
        if Price_change[0,i] > 1:  #强行删去一个很偏离的点
            Price_change[0,i] = 0
    for i in range(1,DIST_PRICE-1):
        Price_change[0,i] = Price[0,i]

    import numpy as np
    Price_change = np.zeros(Price.shape)
    # Price_change[0,1:Y_Price-1] = Price[0,1:Y_Price-1] - Price[0,0:Y_Price-2]
    for i in range(1,Y_Price-1): 
        Price_change[0,i] = Price[0,i] - Price[0,i-1]
        if Price_change[0,i] > 1:  #强行删去一个很偏离的点
            Price_change[0,i] = 0
    Price_change[0,0] = Price[0,0]


    from sklearn.model_selection import train_test_split

    # temp1 =  np.arange(1,Y_totmiun).reshape(1,Y_totmiun-1)#转为1*Y_totmiun-1的矩阵，以使得shape和Total_miu_n匹配
    # temp2 =  np.arange(1,Y_totmiun).reshape(1,Y_totmiun-1)#但这种方式不被train_test_split接受，它要求两个都是数组而不是矩阵
    # X_train,X_test,Y_train,Y_test = train_test_split(temp1[0,0:Y_totmiun-1],temp2[0,0:Y_totmiun-1])

    # X_train,X_test,Y_train,Y_test = train_test_split(Total_miu_n_change[0,0:Y_totmiun-1],Price_change[0,0:Y_Price-1],test_size=0)

    from sklearn.linear_model import LinearRegression

    X_train = Total_miu_n_change[0,0:Y_totmiun-1]
    Y_train = Price_change[0,0:Y_Price-1]
    model=LinearRegression()
    model.fit(X_train.reshape(-1, 1),Y_train.reshape(-1, 1))
    # model.score(X_test.reshape(-1, 1),Y_test.reshape(-1, 1))


    a=model.intercept_#截距
    b=model.coef_#回归系数

    # print(b)

    b_all += b

b_average = b_all / len(SZtick_filenames)
print(b_average)

    


C:\Users\86157\Downloads\quantml-master (1)\quantml-master\data-15.07.08\7.1\000011.SZtick.mat
C:\Users\86157\Downloads\quantml-master (1)\quantml-master\data-15.07.08\7.1\000012.SZtick.mat
C:\Users\86157\Downloads\quantml-master (1)\quantml-master\data-15.07.08\7.1\000029.SZtick.mat
C:\Users\86157\Downloads\quantml-master (1)\quantml-master\data-15.07.08\7.1\000030.SZtick.mat
C:\Users\86157\Downloads\quantml-master (1)\quantml-master\data-15.07.08\7.1\000039.SZtick.mat
C:\Users\86157\Downloads\quantml-master (1)\quantml-master\data-15.07.08\7.10\000011.SZtick.mat
C:\Users\86157\Downloads\quantml-master (1)\quantml-master\data-15.07.08\7.10\000012.SZtick.mat
C:\Users\86157\Downloads\quantml-master (1)\quantml-master\data-15.07.08\7.10\000029.SZtick.mat
C:\Users\86157\Downloads\quantml-master (1)\quantml-master\data-15.07.08\7.10\000030.SZtick.mat
C:\Users\86157\Downloads\quantml-master (1)\quantml-master\data-15.07.08\7.10\000039.SZtick.mat
C:\Users\86157\Downloads\quantml-master (1)\q

C:\Users\86157\AppData\Local\Temp\ipykernel_3996\3669097000.py:43: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  miu_n = BidVolume.astype(np.int) - AskVolume.astype(np.int)
C:\Users\86157\AppData\Local\Temp\ipykernel_3996\3669097000.py:44: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use

[[2.99406666e-08]]


C:\Users\86157\AppData\Local\Temp\ipykernel_3996\3669097000.py:44: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  Total_miu_n = TotalBidVolume.astype(np.int) - TotalAskVolume.astype(np.int)
